In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from simple_pipeline import simple_pipeline

In [ ]:
from complex_xgboost import complex_XGBoost_model

In [ ]:
df = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/train.csv', parse_dates=['date']).drop('row_id',axis=1)
print(df.shape)
df.head()

In [ ]:
target = 'num_sold'
modeltype = 'Regression'

In [ ]:
if isinstance(target, str):
    cols = [x for x in list(df) if x not in [target]]
else:
    cols = [x for x in list(df) if x not in target]
X = df[cols]
y = df[target]
df.dtypes

In [ ]:
###  find the y values that have low samples ####
if modeltype != 'Regression':
    low_counts = y.value_counts()[(y.value_counts()<=1).values].index
    print(len(low_counts))
    ## You need to remove those rows that have just one sample ##
    X = X[~(y.isin(low_counts))]
    y = y[~(y.isin(low_counts))]
X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split
if modeltype == 'Regression':
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1, random_state=999)
else:
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,stratify=y, random_state=999)
print(X_train.shape, X_test.shape)

In [ ]:
### This returns a pipeline ## you need to fit it to see how well it performs on a hold out dataset
smp, mlp = simple_pipeline(X_train, y_train, target, with_model=False, date_to_string=False)

from sklearn import set_config
#### You can display your pipeline in a nice diagram if it is a Jupyter Notebook ##
set_config(display="diagram")
mlp

In [ ]:
X_XGB = smp.fit_transform(X)
print(X_XGB.shape)
X_XGB.head(1)

In [ ]:
test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/test.csv', parse_dates=['date']).drop('row_id',axis=1)
print(test.shape)
test.head()

In [ ]:
X_XGB_test = smp.transform(test)
print(X_XGB_test.shape)
X_XGB_test.head(1)

In [ ]:
outputs = complex_XGBoost_model(x_train=X_XGB, y_train=y, x_test=X_XGB_test, log_y=False, 
                                GPU_flag=False, scaler='', enc_method='', n_splits=5, verbose=-1)

In [ ]:
y_pred = outputs[0]
y_pred.shape

from sklearn.metrics import balanced_accuracy_score, mean_squared_error
if modeltype == 'Regression':
    print(np.sqrt(mean_squared_error(y_test, mlp.predict(X_test))))
else:
    print(balanced_accuracy_score(y_test, mlp.predict(X_test)))

In [ ]:
subm = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/sample_submission.csv')
print(subm.shape)
subm.head()

In [ ]:
subm[target] = y_pred
subm.head()

In [ ]:
subm.to_csv('submission.csv', index=False)